In [2]:
import cv2
from deepface import DeepFace
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import numpy as np
import threading

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import tkinter as tk
from threading import Thread
import time
import os

class MoodDetectionSystem:
    def __init__(self):
        # Initialize the UI window
        self.root = tk.Tk()
        self.root.title("Mood Detection System")
        self.root.geometry("800x600")

        # Load pretrained face detection
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
        
        # Load the emotion model from the local directory
        model_path = os.path.join(os.path.dirname(D:\Users\Home_Desktop\Desktop\Mj\PROJECTS\Expression-X\facial_expression_model_weights.h5), 'facial_expression_model_weights.h5')
        self.emotion_model = load_model(model_path)
        
        # Emotion labels matching the deepface facial expression model
        self.emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']
        
        # State variables
        self.is_running = False
        self.video_capture = None
        
        # UI Elements
        self.canvas = tk.Canvas(self.root, width=640, height=480)
        self.canvas.pack(pady=20)
        
        self.start_button = tk.Button(self.root, text="Start Detection", command=self.start_detection)
        self.start_button.pack(side=tk.LEFT, padx=50)
        
        self.stop_button = tk.Button(self.root, text="Stop Detection", command=self.stop_detection)
        self.stop_button.pack(side=tk.RIGHT, padx=50)
        
        self.status_label = tk.Label(self.root, text="Status: Stopped")
        self.status_label.pack(pady=10)

    def preprocess_face(self, face):
        # Preprocess the face image for the specific deepface model
        face = cv2.resize(face, (48, 48))  # Size expected by this model
        face = cv2.cvtColor(face, cv2.COLOR_GRAY2RGB)
        face = img_to_array(face)
        face = np.expand_dims(face, axis=0)
        face = face / 255.0  # Normalization
        return face

    def detect_mood(self):
        while self.is_running:
            ret, frame = self.video_capture.read()
            if not ret:
                break

            # Convert to grayscale for face detection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)

            for (x, y, w, h) in faces:
                # Draw rectangle around face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                
                # Extract face ROI
                face_roi = gray[y:y+h, x:x+w]
                
                # Preprocess for emotion detection
                processed_face = self.preprocess_face(face_roi)
                
                # Predict emotion using the loaded model
                emotion_pred = self.emotion_model.predict(processed_face)
                emotion_idx = np.argmax(emotion_pred)
                emotion = self.emotion_labels[emotion_idx]
                
                # Display emotion
                cv2.putText(frame, emotion, (x, y-10), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Convert frame to RGB for Tkinter
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = tk.PhotoImage(data=cv2.imencode('.png', frame_rgb)[1].tobytes())
            self.canvas.create_image(0, 0, anchor=tk.NW, image=img)
            self.canvas.image = img
            
            time.sleep(0.03)  # Control frame rate

        self.video_capture.release()
        cv2.destroyAllWindows()

    def start_detection(self):
        if not self.is_running:
            self.is_running = True
            self.video_capture = cv2.VideoCapture(0)  # 0 for default webcam
            self.status_label.config(text="Status: Running")
            Thread(target=self.detect_mood, daemon=True).start()

    def stop_detection(self):
        self.is_running = False
        self.status_label.config(text="Status: Stopped")

    def run(self):
        self.root.mainloop()

    def __del__(self):
        if self.video_capture is not None:
            self.video_capture.release()

if __name__ == "__main__":
    # Create and run the mood detection system
    detector = MoodDetectionSystem()
    detector.run()

NameError: name '__file__' is not defined